# Informations sur les fournisserus de soins

Données
* Base de personnels de soin: `ps-infospratiques.csv` (annuaire de la CNAM) [https://www.data.gouv.fr/fr/datasets/annuaire-sante-de-la-cnam/]
* Base de correspondance entre CP et code commune INSEE [https://www.data.gouv.fr/fr/datasets/r/554590ab-ae62-40ac-8353-ee75162c05ee]
 


In [9]:
import pandas as pd
import numpy as np

In [10]:
data = pd.read_csv("../data/ps-infospratiques.csv", sep=";", header=None, encoding="latin_1")
data.rename(columns={
                0:"Sexe", 
                1:"nom", 
                2:"prenom", 
                3:"Adresse ligne 1,", 
                4:"Adresse ligne 2",
                5:"Adresse ligne 3",
                6:"Adresse ligne 4",
                7:"CP", ## un numéro qui sembl être un CP (mais des soucis avec les 2 ... (?))
                8:"Ville",
                9:"Téléphone", 
                10:"Profession", #code de profession (voir nommenclature ci-dessous)
                11:"mode", #mode d'exercice
                12:"nature", #nature d'exercice
                13:"convention", 
                14:"option contrat", 
                15:"sesam_vital", 
                16:"Type", # Liberal/hors activité, etc.
                17:"Type consultation", 
                18:"heure_debut", 
                19:"heure_debut",
                20:"jour"}, inplace=True)

/home/tguyet/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
#data = pd.read_csv("../data/ps-infospratiques.csv", sep=";", header=None, encoding="latin_1", usecols=[0,3,4,5,6,7,8,10,11,12,16])
#data.rename(columns={
#                0:"Sexe",  
#                3:"Adresse ligne 1,", 
#                4:"Adresse ligne 2",
#                5:"Adresse ligne 3",
#                6:"Adresse ligne 4",
#                7:"CP", ## un numéro qui sembl être un CP (mais des soucis avec les 2 ... (?))
#                8:"Ville",
#                10:"Profession", #code de profession (voir nommenclature ci-dessous)
#                11:"mode", #mode d'exercice
#                12:"nature", #nature d'exercice
#                16:"Type", # Liberal/hors activité, etc.
#            }, inplace=True)

In [12]:
data.head()

,Sexe,nom,prenom,"Adresse ligne 1,",Adresse ligne 2,Adresse ligne 3,Adresse ligne 4,CP,Ville,Téléphone,...,mode,nature,convention,option contrat,sesam_vital,Type,Type consultation,heure_debut,heure_debut,jour
0,F,CUGNIET,ODILE,NaN,NaN,35 RUE CENTRALE,NaN,1230,TENAY,NaN,...,NaN,3,c1,N,O,NaN,NaN,NaN,NaN,NaN
1,H,TANAZACQ,REMI,NaN,NaN,1 ROUTE DE CHAMOISE,NaN,1430,ST MARTIN DU FRENE,NaN,...,NaN,3,c1,O,O,NaN,NaN,NaN,NaN,NaN
2,H,HOJABR,AMIR THEODORE,NaN,NaN,76 RUE GRANDE,NaN,1220,DIVONNE LES BAINS,971169655.0,...,NaN,3,c3,N,O,NaN,NaN,NaN,NaN,NaN
3,H,HOJABR,AMIR THEODORE,NaN,CLINIQUE CENDANEG,180 ROUTE DU NANT,NaN,1280,PREVESSIN MOENS,NaN,...,NaN,3,c3,N,O,NaN,NaN,NaN,NaN,NaN
4,F,ADAM,EMILIE,NaN,NaN,180 ROUTE DU NANT,NaN,1280,PREVESSIN MOENS,NaN,...,NaN,3,c3,N,O,NaN,NaN,NaN,NaN,NaN


In [13]:
# Médecins libéraux exercant en ville
medecins=data[(data['Type']!=0) & (data['Type']!=6) & (data['Type']!=7) & (data['Type']!=8) ]

# codage du sex selon la IR_SEX_COD
medecins["Sexe"].fillna("9",inplace=True)
medecins["Sexe"]=medecins["Sexe"].str.replace(pat="F",repl="2")
medecins["Sexe"]=medecins["Sexe"].str.replace(pat="H",repl="1")
# suppression des CEDEX
medecins["Ville"]=medecins["Ville"].str.replace(pat=" CEDEX.*",repl="", regex=True)

medecins=medecins[["Sexe","nom","prenom","CP","Ville","Profession"]].drop_duplicates()
# on supprime les "non-médecins" (parce qu'ils ne correspondent pas à cette classe des spécialistes)
medecins=medecins[ medecins["Profession"]!=1 ] # suppression dans ambulanciers
medecins=medecins[ medecins["Profession"]!=2 ] # suppression dans anathamo-patho
medecins=medecins[ (medecins["Profession"]<18) | (medecins["Profession"]>21) ] # suppression des dentistes
medecins=medecins[ (medecins["Profession"]<24) | (medecins["Profession"]>32) ] # suppression des fournisseurs de matériel
medecins=medecins[ (medecins["Profession"]!=39) ] # suppression des infirmiers
medecins=medecins[ (medecins["Profession"]<40) | (medecins["Profession"]>42)] # suppression des laboratoires
medecins=medecins[ (medecins["Profession"]!=43) ] # suppression des kinés
medecins=medecins[ (medecins["Profession"]!=57) ] # suppression des orthophonistes
medecins=medecins[ (medecins["Profession"]!=58) ] # suppression des orthoptiste
medecins=medecins[ (medecins["Profession"]!=58) ] # suppression des pédicures-podologues
medecins=medecins[ (medecins["Profession"]<62) | (medecins["Profession"]>63)] # suppression des pharmaciens
medecins=medecins[ (medecins["Profession"]!=71) ] # suppression des sages-femmes

medecins.drop(["nom","prenom"],axis=1, inplace=True)
medecins

/home/tguyet/.local/lib/python3.8/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,Sexe,CP,Ville,Profession
0,2,1230,TENAY,45
1,1,1430,ST MARTIN DU FRENE,6
2,1,1220,DIVONNE LES BAINS,56
3,1,1280,PREVESSIN MOENS,56
4,2,1280,PREVESSIN MOENS,15
...,...,...,...,...
1176601,2,75015,PARIS,65
1176607,2,13007,MARSEILLE,67
1176608,1,40230,ST VINCENT DE TYROSSE,6
1176614,1,89480,COULANGES SUR YONNE,6


In [14]:
medecins.head()

,Sexe,CP,Ville,Profession
0,2,1230,TENAY,45
1,1,1430,ST MARTIN DU FRENE,6
2,1,1220,DIVONNE LES BAINS,56
3,1,1280,PREVESSIN MOENS,56
4,2,1280,PREVESSIN MOENS,15


## Get code commune Insee

In [7]:
data = pd.read_csv("../data/laposte_hexasmal.csv", sep=";", usecols=["Code_commune_INSEE","Code_postal","Nom_commune"])

In [12]:
# du fait des libellés d'acheminement, il peut y avoir plusieurs entrées identique pour la correspondance CP/INSEE
data.drop_duplicates(inplace=True)
data

,Code_commune_INSEE,Nom_commune,Code_postal,dpt
0,70109,BUTHIERS,70190,70
1,70111,CALMOUTIER,70240,70
2,70118,CHAMBORNAY LES BELLEVAUX,70190,70
3,70121,CHAMPEY,70400,70
4,70124,CHAMPTONNAY,70100,70
...,...,...,...,...
39182,50148,COUVAINS,50680,50
39183,50161,LE DEZERT,50620,50
39184,50162,DIGOSVILLE,50110,50
39186,50172,EMONDEVILLE,50310,50


In [14]:
medecins_insee=pd.merge(data, medecins, how="right", right_on=["CP","Ville"], left_on=["Code_postal","Nom_commune"])[["Code_commune_INSEE","Ville","CP","Sexe","Profession"]]
medecins_insee.head()

,Code_commune_INSEE,Ville,CP,Sexe,Profession
0,70197,DAMPIERRE SUR LINOTTE,70230,1,45
1,70197,DAMPIERRE SUR LINOTTE,70230,1,45
2,70205,ECHAVANNE,70400,2,45
3,70207,ECHENOZ LA MELINE,70000,2,45
4,70207,ECHENOZ LA MELINE,70000,1,45


In [15]:
##même taille que la liste initial des médecins ...
print(len(medecins)) 
print(len(medecins_insee)) 

160409


In [18]:
#mais ... des correspondances n'ont pas fontionner
medecins_manquants=medecins_insee[medecins_insee["Code_commune_INSEE"].fillna("")==""]
medecins_manquants

,Code_commune_INSEE,Ville,CP,Sexe,Profession
126318,NaN,ST MARTIN DU FRESNE,1430,1,6
126319,NaN,ST MARTIN DU FRESNE,1430,1,45
126320,NaN,ST MARTIN DU FRESNE,1430,1,52
126321,NaN,ST MARTIN DU FRESNE,1430,2,45
126322,NaN,ST MARTIN DU FRESNE,1430,1,6
...,...,...,...,...,...
160404,NaN,SEVRIER,74330,2,56
160405,NaN,ST AUBIN DE BAUBIGNE,79700,1,45
160406,NaN,HYERES,83407,1,67
160407,NaN,CAPESTERRE B EAU,97130,2,45


Problemes dans les codages de la base des médecins a priori
* probleme dans certains noms de communes (FRESNE a la place de FRENE par exemple)
* probleme également dans ces CP

La solution ... est d'améliorer le matching des communes dans un premier temps

In [26]:
#utilisation d'une librairie permettant de faire une matching approximatif
import difflib

# Pour chaque ville, on cherche la correspondance la plus proche dans la base des communes disponibles !
## ATTENTION: cette étape est longue en calcul
def proceed(x):
    corresp = difflib.get_close_matches(x, data["Nom_commune"])
    if len(corresp)==0:
        return ""
    else:
        return corresp[0]
    return 

#medecins_manquants['ville_align']=medecins_manquants["Ville"].apply(proceed)

<ipython-input-26-9f090416dbd7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medecins_manquants['ville_align']=medecins_manquants["Ville"].apply(proceed)


In [27]:
medecins_manquants

,Code_commune_INSEE,Ville,CP,Sexe,Profession,ville_align
126318,NaN,ST MARTIN DU FRESNE,1430,1,6,ST MARTIN DU FRENE
126319,NaN,ST MARTIN DU FRESNE,1430,1,45,ST MARTIN DU FRENE
126320,NaN,ST MARTIN DU FRESNE,1430,1,52,ST MARTIN DU FRENE
126321,NaN,ST MARTIN DU FRESNE,1430,2,45,ST MARTIN DU FRENE
126322,NaN,ST MARTIN DU FRESNE,1430,1,6,ST MARTIN DU FRENE
...,...,...,...,...,...,...
160404,NaN,SEVRIER,74330,2,56,SEVRIER
160405,NaN,ST AUBIN DE BAUBIGNE,79700,1,45,ST AUBIN D AUBIGNE
160406,NaN,HYERES,83407,1,67,HYERES
160407,NaN,CAPESTERRE B EAU,97130,2,45,CAPESTERRE BELLE EAU


Ensuite, on peut recommencer à faire l'alignement, mais sur la ville normalisée ....

In [ ]:
medecins_insee=pd.merge(data, medecins, how="right", right_on=["CP","ville_align"], left_on=["Code_postal","Nom_commune"])[["Code_commune_INSEE","Ville","CP","Sexe","Profession"]]
medecins_insee.head()

In [119]:
#encore quelques médecins manquants à cause des CP
print( len(medecins_insee[medecins_insee["Code_commune_INSEE"].fillna("")==""]) )
print( len(medecins_insee[medecins_insee["Code_commune_INSEE"].fillna("")==""])/len(medecins_insee) )

30899
0.19262274642794802


In [121]:
medecins_insee[medecins_insee["Code_commune_INSEE"].fillna("")!=""].to_csv("medecins.csv")

## On recommence ...

Avec les nouvelles communes, certains noms de communes semblent être indiqués dans le champs "Ligne 5"

In [79]:
data = pd.read_csv("../data/laposte_hexasmal.csv", sep=";")

In [80]:
def fusion(x):
    if pd.isnull(x['Ligne_5']):
        return x['Nom_commune']
    else:
        return x['Ligne_5']
data['Nom_commune']=data.apply(fusion,axis=1)
data=data[["Code_commune_INSEE","Nom_commune","Code_postal"]]
data.drop_duplicates(inplace=True)
data

,Code_commune_INSEE,Nom_commune,Code_postal
0,70109,BUTHIERS,70190
1,70111,CALMOUTIER,70240
2,70118,CHAMBORNAY LES BELLEVAUX,70190
3,70121,CHAMPEY,70400
4,70124,CHAMPTONNAY,70100
...,...,...,...
39187,50188,LA BESLIERE,50320
39188,50188,LE MESNIL DREY,50320
39189,50197,LE MESNIL BONANT,50450
39190,50216,GRAIGNES MESNIL ANGOT,50620


In [81]:
medecins_insee=pd.merge(data, medecins, how="right", right_on=["CP","Ville"], left_on=["Code_postal","Nom_commune"])[["Code_commune_INSEE","Ville","CP","Sexe","Profession"]]
medecins_insee.head()

,Code_commune_INSEE,Ville,CP,Sexe,Profession
0,70197,DAMPIERRE SUR LINOTTE,70230,1,45
1,70197,DAMPIERRE SUR LINOTTE,70230,1,45
2,70205,ECHAVANNE,70400,2,45
3,70207,ECHENOZ LA MELINE,70000,2,45
4,70207,ECHENOZ LA MELINE,70000,1,45


In [82]:
##même taille que la liste initial des médecins ...
print(len(medecins)) 
print(len(medecins_insee)) 

160409
160412


In [83]:
#mais ... des correspondances n'ont pas fontionner
medecins_manquants=medecins_insee[medecins_insee["Code_commune_INSEE"].fillna("")==""]
medecins_manquants

,Code_commune_INSEE,Ville,CP,Sexe,Profession
129513,NaN,ST MARTIN DU FRESNE,1430,1,6
129514,NaN,ST MARTIN DU FRESNE,1430,1,45
129515,NaN,ST MARTIN DU FRESNE,1430,1,52
129516,NaN,ST MARTIN DU FRESNE,1430,2,45
129517,NaN,ST MARTIN DU FRESNE,1430,1,6
...,...,...,...,...,...
160407,NaN,SEVRIER,74330,2,56
160408,NaN,HYERES,83407,1,67
160409,NaN,LES SABLES D OLONNE,85340,2,70
160410,NaN,CAPESTERRE B EAU,97130,2,45


In [84]:
#utilisation d'une librairie permettant de faire une matching approximatif
import difflib

# Pour chaque ville, on cherche la correspondance la plus proche dans la base des communes disponibles !
## ATTENTION: cette étape est longue en calcul -> appliqué sur les unique()
def proceed(x):
    corresp = difflib.get_close_matches(x, data["Nom_commune"])
    if len(corresp)==0:
        return ""
    else:
        return corresp[0]

rename=pd.Series(medecins_manquants ["Ville"].unique()).apply(proceed)

In [85]:
corresp=pd.concat([pd.Series(medecins_manquants["Ville"].unique()),rename], axis=1)
corresp.rename(columns={0:"OldVille",1:"Ville"},inplace=True)
corresp

,OldVille,Ville
0,ST MARTIN DU FRESNE,ST MARTIN DU FRENE
1,BOURG EN BRESSE,BOURG EN BRESSE
2,OYONNAX,OYONNAX
3,AMBERIEU EN BUGEY,AMBERIEU EN BUGEY
4,TREFFORT VAL REVERMONT,ROCHEFORT SUR BREVON
...,...,...
1278,CHARBONNIERES,CHARBONNIERES
1279,SEVRIER,SEVRIER
1280,HYERES,HYERES
1281,CAPESTERRE B EAU,CAPESTERRE BELLE EAU


In [94]:
len(corresp['OldVille'].unique())

1283

In [88]:
medecins_manquants_filled=pd.merge(medecins_manquants, corresp, left_on="Ville", right_on="OldVille", suffixes=("old",""))[["Ville","OldVille","CP","Sexe",'Profession']]
medecins_manquants_filled

,Ville,OldVille,CP,Sexe,Profession
0,ST MARTIN DU FRENE,ST MARTIN DU FRESNE,1430,1,6
1,ST MARTIN DU FRENE,ST MARTIN DU FRESNE,1430,1,45
2,ST MARTIN DU FRENE,ST MARTIN DU FRESNE,1430,1,52
3,ST MARTIN DU FRENE,ST MARTIN DU FRESNE,1430,2,45
4,ST MARTIN DU FRENE,ST MARTIN DU FRESNE,1430,1,6
...,...,...,...,...,...
30894,CHARBONNIERES,CHARBONNIERES,69260,2,45
30895,SEVRIER,SEVRIER,74330,2,56
30896,HYERES,HYERES,83407,1,67
30897,CAPESTERRE BELLE EAU,CAPESTERRE B EAU,97130,2,45


In [89]:
medecins_insee2=pd.merge(data, medecins_manquants_filled, how="right", left_on=["Code_postal","Nom_commune"], right_on=["CP","Ville"])[["Code_commune_INSEE","Ville","OldVille","CP","Sexe","Profession"]]
medecins_insee2.head()
len(medecins_insee2)

30899

In [95]:
#mais ... il en reste toujours sans correspondance !!
medecins_manquants2=medecins_insee2[medecins_insee2["Code_commune_INSEE"].fillna("")==""]
medecins_manquants2

,Code_commune_INSEE,Ville,OldVille,CP,Sexe,Profession
724,NaN,BOURG EN BRESSE,BOURG EN BRESSE,1012,1,7
725,NaN,BOURG EN BRESSE,BOURG EN BRESSE,1012,1,37
726,NaN,BOURG EN BRESSE,BOURG EN BRESSE,1012,1,64
727,NaN,BOURG EN BRESSE,BOURG EN BRESSE,1012,1,12
728,NaN,BOURG EN BRESSE,BOURG EN BRESSE,1012,1,23
...,...,...,...,...,...,...
30894,NaN,LIGNY EN CAMBRESIS,INCHY EN CAMBRESIS,59540,1,45
30895,NaN,LE BOUSQUET,LE TOUQUET,62520,1,45
30896,NaN,CHARBONNIERES,CHARBONNIERES,69260,2,45
30897,NaN,SEVRIER,SEVRIER,74330,2,56


In [114]:
medecins_manquants_filled['dpt']=medecins_manquants2['CP']//1000
data['dpt']=data['Code_postal']//1000
medecins_insee2_villedpt=pd.merge(data[["Code_commune_INSEE",'dpt',"Nom_commune"]].drop_duplicates(), 
                                  medecins_manquants_filled, 
                                  how="right", left_on=['dpt',"Nom_commune"], right_on=['dpt',"Ville"])[["Code_commune_INSEE","Ville","OldVille","CP","Sexe","Profession"]]
len(medecins_insee2_ville)

43196

In [112]:
print(len(data[['dpt',"Nom_commune"]]))
print(len(data[["Code_commune_INSEE",'dpt',"Nom_commune"]].drop_duplicates()))
print(len(data.drop_duplicates()))
data

39078
38920
39078


,Code_commune_INSEE,Nom_commune,Code_postal,dpt
0,70109,BUTHIERS,70190,70
1,70111,CALMOUTIER,70240,70
2,70118,CHAMBORNAY LES BELLEVAUX,70190,70
3,70121,CHAMPEY,70400,70
4,70124,CHAMPTONNAY,70100,70
...,...,...,...,...
39187,50188,LA BESLIERE,50320,50
39188,50188,LE MESNIL DREY,50320,50
39189,50197,LE MESNIL BONANT,50450,50
39190,50216,GRAIGNES MESNIL ANGOT,50620,50


In [102]:
medecins_insee2_ville=pd.merge(data, medecins_manquants_filled, how="right", left_on=["Nom_commune"], right_on=["CP","Ville"])[["Code_commune_INSEE","Ville","OldVille","CP","Sexe","Profession"]]
len(medecins_insee2_ville)

43196

In [103]:
medecins_manquants=medecins_insee_ville[medecins_insee_ville["Code_commune_INSEE"].fillna("")==""]
medecins_manquants

,Code_commune_INSEE,Ville,CP,Code_postal,Sexe,Profession
43191,NaN,,99131,NaN,1,45
43192,NaN,,99131,NaN,1,45
43193,NaN,,99131,NaN,1,45
43194,NaN,,99131,NaN,1,45
43195,NaN,,99131,NaN,2,45


In [104]:
np.sum( (medecins_insee_ville['CP']-medecins_insee_ville['Code_postal']) <=1000 )

28563

In [105]:
medecins_insee_ville[medecins_insee_ville['CP']-medecins_insee_ville['Code_postal']>1000]

,Code_commune_INSEE,Ville,CP,Code_postal,Sexe,Profession
15,71582,MASSY,91349,71250.0,2,51
17,76415,MASSY,91349,76270.0,2,51
258,72296,ST MAIXENT,79400,72320.0,1,67
466,79128,FRANCOIS,97240,79260.0,1,45
467,79128,FRANCOIS,97240,79260.0,1,35
...,...,...,...,...,...,...
43083,44094,MAUVES SUR LOIRE,49750,44470.0,1,45
43084,44094,MAUVES SUR LOIRE,49620,44470.0,1,61
43085,44094,MAUVES SUR LOIRE,49410,44470.0,2,45
43086,45201,MERINVILLE,54370,45210.0,1,45


## Nouveaux test avec une autre base !!

Base ODS [https://www.data.gouv.fr/fr/datasets/r/557f25da-4bfd-4306-a342-9bd2adb49ba9] datant de 2014

SyntaxError: invalid syntax (<ipython-input-53-68e33824fa78>, line 1)